In [ ]:
# There, done. That was easy.
# That's all we needed to do, right?
import numpy as np
import pandas as pd
import xgboost as xgb

import json
import re

In [ ]:
# Read in data
FIRST = 5 # First year is 2015
LAST = 7 # Last year is 2017

raw_data = [pd.read_csv('data/201%d.csv' % i) for i in xrange(FIRST, LAST + 1)]

years = np.array([np.repeat('201%d' % (i + FIRST,), raw_data[i].shape[0])
                  for i in xrange(0, LAST - FIRST + 1)])

# raw_data eventually combined into columns in data
data = pd.DataFrame({'year': np.hstack(years)})

In [ ]:
# helpers to define how to get columns
number = re.compile('\d+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

def get_satisfaction(response):
    if type(response) == 'float64':
        return response

    if response in satisfaction_strs:
        return satisfaction_strs[response]

    return np.nan

In [ ]:
# define how to get each column
colnames = [
    {
        'new_name': 'jobSatisfaction',
        'old_names': [
            'Job Satisfaction',
            'job_satisfaction',
            'JobSatisfaction',
        ],
        'apply': np.vectorize(get_satisfaction),
    },
    {
        'new_name': 'experience',
        'old_names': [
            'Years IT / Programming Experience',
            'experience_range',
            'YearsProgram',
        ],
        'apply': np.vectorize(get_first_number),
    },
    {
        'new_name': 'salary',
        'old_names': [
            'Compensation: midpoint',
            'salary_midpoint',
            'Salary'
        ]
    },
]

In [ ]:
# Take previous cell and use information to make the "data" dataframe
def get_col(data, coldata):
    new_col = np.array([dataset[colname]
                        for dataset, colname in zip(data, coldata['old_names'])])
    if 'apply' in coldata:
        new_col = coldata['apply'](new_col)
    return np.hstack(new_col)

new_cols = {row['new_name']: get_col(raw_data, row) for row in colnames}
data = data.assign(**new_cols)